# Final Project, COMS 4995_005, Deep Learning

## Team Information

Team Member1 (Name,UNI): Richard Kennedy, rk2860

Python Version: 2
Utilizing Pytorch

In [1]:
import torch
import cPickle
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchtext.vocab as Vocab

import torchvision.datasets as dset
import torchvision.transforms as transforms

import PIL

In [17]:
cap = dset.CocoCaptions(root = './train2014',
                        annFile = 'annotations/captions_train2014.json',
                        transform= transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()]))

res = models.resnet50(pretrained=True)
vdim = 100
rSize = 2048
x = Vocab.GloVe(name='6B', dim= vdim)

paddSize =  rSize - vdim



loading annotations into memory...
Done (t=2.52s)
creating index...
index created!


100%|██████████| 400000/400000 [00:51<00:00, 7714.52it/s]


In [18]:
a = x.vectors[x.stoi['man']]
print a.size()
b = torch.FloatTensor(paddSize).zero_()
paddedVec = torch.cat((a, b), 0)

torch.Size([100])


In [19]:
print "HELLO"
# img, target = cap[0]
# words = ['StopLSTM'] + target[0].split(' ') + ['EndLSTM']
# print words


# print wordVec(Vocab[words[1]])

HELLO


## Part 1

In [ ]:
class LSTM(nn.Module):
    def __init__(self, embed_size, hidden_size, res50):
        super(LSTM, self).__init__()
        
        self.Esize = embed_size
        self.Hsize = hidden_size
        self.Wix = nn.Linear(embed_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)
        self.Wfx = nn.Linear(embed_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)
        self.Wox = nn.Linear(embed_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)
        self.Wcx = nn.Linear(embed_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)
        
        #res50 = torchvision.models.resnet50(pretrained=True)
        self.res50NoGrad = torch.nn.Sequential(*list(res50.children())[:-2])
        #Every layer besides the last layer does not need to be trained
        for param in self.res50NoGrad.parameters():
            param.requires_grad = False
        
        #set embed_size to dimensions of self.res50Last
        #output should be second to last layer
        self.res50Last = torch.nn.Sequential(list(res50.children())[-2])
        
        
        #self.glove = Vocab.GloVe(name='840B', dim= embed_size)
        
        
        
        

    def forward(self, x, m = None, c = None, CNN = False ):
        
        if not CNN:
            #glove requires x to be one hot word encoding?
            #x = self.glove(x)
            i = F.sigmoid(self.Wix(x)+self.Wim(m))
            f = F.sigmoid(self.Wfx(x)+self.Wfm(m))
            o = F.sigmoid(self.Wox(x)+self.Wom(m))
            h = F.tanh(self.Wcx(x) + self.Wcm(m))
            newC = f*c + i*h
            newM = o*newC
            #prediction dimension should line up with vocab_size
            #fine tune hidden dimension to make that work
            
            #log_softmax might not be necessary -> due to backprop
            
            #calculations: hidden dimensions = 50, embed_size = 2048, vocab_size not used
            #prediction = F.log_softmax(newM,)
            
            return newM, newC, newM
        
        else:
            x = self.res50NoGrad(x)
            x = self.res50Last(x)
            x = x.resize(1, self.Esize)
            i = F.sigmoid(self.Wix(x))
            f = F.sigmoid(self.Wfx(x))
            o = F.sigmoid(self.Wox(x))
            h = F.tanh(self.Wcx(x))
            c = i*h
            m = o*c
            
            return None, c, m

# net = LSTM(2048, 50, res)
# img, target = cap[0]
# img = img.unsqueeze(0)
# junk, c0, m0 =  net.forward(Variable(img), CNN = True)

# a = x.vectors[x.stoi['man']]
# b = torch.FloatTensor(1998).zero_()
# paddedVec = torch.cat((a, b), 0)

# pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
# pred = pred.unsqueeze(0)

## Part 2

In [ ]:
##Train

startW = 'start'
endW = 'stop'
## was lazy and used cat and dog as start and stop word. 
## Definetely going to have to change that
net = LSTM(rSize, vdim, res)
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad,net.parameters()), lr=0.001, momentum=.3)

counter = 0

for epoch in range(0, 1000):
    logLoss = 0.0
    
    torch.save(net, str(epoch)+'LSTM.pt')
    for i in range(0, len(cap)):
        img, target = cap[i]
        words = [startW] + target[0].split(' ') + [endW]
        length = len(words)
        img = img.unsqueeze(0)
        junk, c0, m0 =  net.forward(Variable(img), CNN = True)
        fullPred = None
        fullAnsw = None
        flip = True
        for j in range(0, length-1):
            #Going to need larger word corpus
            if words[j].lower() in x.stoi and words[j+1].lower() in x.stoi:

                a = x.vectors[x.stoi[words[j].lower()]]
                b = torch.FloatTensor(paddSize).zero_()
                paddedVec = torch.cat((a, b), 0)
                pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)

                #temp = torch.FloatTensor(400000).zero_()
                #temp[x.stoi[words[j+1].lower()]] = 1

                answer = x.vectors[x.stoi[words[j+1].lower()]]
                answer = answer.unsqueeze(0)




                if flip:
                    fullPred = pred
                    fullAnsw = answer.unsqueeze(0)
                    flip = False
                else:
                    answer = answer.unsqueeze(0)
                    fullPred = torch.cat((fullPred, pred), 0)
                    fullAnsw = torch.cat((fullAnsw, answer), 0)


        
        loss = criterion(fullPred, Variable(fullAnsw))
        loss.backward()
        optimizer.step()

        logLoss += loss.data[0]
        if counter % 100 == 99:    
            print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss/100))
            logLoss = 0.0

        counter+=1   
    
    

EPOCH: 0, LOSS: 0.300906306654
EPOCH: 0, LOSS: 0.210703996718
EPOCH: 0, LOSS: 0.205358923376
EPOCH: 0, LOSS: 0.204190388471
EPOCH: 0, LOSS: 0.192558823675
EPOCH: 0, LOSS: 0.186805103868
EPOCH: 0, LOSS: 0.186626527756
EPOCH: 0, LOSS: 0.187759270966


## Extra Credit